# Markov Computations

This notebook provides the computations contained in the book along with the examples of their use.

If you want to import this into a running Julia shell, first choose "Download as" on the "File" menu and select "Julia (.jl)".  Next, start a Julia shell and include the file.

```juila-repl
julia> include("Markov Computations.jl")
```

This may take a minute or two, and will display the variables  and functions that are now available at the end.

## Contents
From Chapter 1 of the text
  * [Preliminaries](#Preliminaries)
  * [Representing models](#Representing-models)

Note that we will use the `LinearAlgebra` package.

In [1]:
using LinearAlgebra

## Preliminaries

The following is a procedure to row-normalize a matrix, all of whose entries are assumed to be nonnegative numbers.

In [2]:
"""
Row normalize a matrix of nonnegative values to
produce a row-stochastic matrix.  The input
matrix cannot have any row summing to zero!

# Examples
```julia-repl
julia> P = row_norm(A)
```
"""
function row_norm(A::Array)
    return A .* (1 ./ (A*ones(size(A)[2])))
end

row_norm

In [3]:
A = [ 0 1 0 0 0
      0 0 1 1 0
      0 0 4 2 2
      0 1 0 0 3
      1 0 0 0 0 ]
P = row_norm(A)

5×5 Array{Float64,2}:
 0.0  1.0   0.0  0.0   0.0
 0.0  0.0   0.5  0.5   0.0
 0.0  0.0   0.5  0.25  0.25
 0.0  0.25  0.0  0.0   0.75
 1.0  0.0   0.0  0.0   0.0

## Representing models

The following is the single-step transition matrix from the example.

$$ P = \left[
       \begin{matrix}
       0 & 1 & 0 & 0 & 0 \\
       0 & 0 & \frac{1}{2} & \frac{1}{2} & 0 \\
       0 & 0 & \frac{1}{2} & \frac{1}{4} & \frac{1}{4} \\
       0 & \frac{1}{4} & 0 & 0 & \frac{3}{4} \\
       1 & 0 & 0 & 0 & 0 
       \end{matrix}
       \right] $$

In [4]:
P = [ 0    1    0    0    0
      0    0  1//2 1//2   0
      0    0  1//2 1//4 1//4
      0  1//4   0    0  3//4
      1    0    0    0    0 ]

5×5 Array{Rational{Int64},2}:
 0//1  1//1  0//1  0//1  0//1
 0//1  0//1  1//2  1//2  0//1
 0//1  0//1  1//2  1//4  1//4
 0//1  1//4  0//1  0//1  3//4
 1//1  0//1  0//1  0//1  0//1

An alternate way to define the matrix is to use frequency or other weights and row-normalize.

In [5]:
row_norm([0 1 0 0 0; 0 0 1 1 0; 0 0 2 1 1; 0 1 0 0 3; 1 0 0 0 0])

5×5 Array{Float64,2}:
 0.0  1.0   0.0  0.0   0.0
 0.0  0.0   0.5  0.5   0.0
 0.0  0.0   0.5  0.25  0.25
 0.0  0.25  0.0  0.0   0.75
 1.0  0.0   0.0  0.0   0.0

The following is the reduced transition matrix (the single-step matrix minus the row and column for the sink).

$$ Q = \left[
       \begin{matrix}
       0 & 1 & 0 & 0 \\
       0 & 0 & \frac{1}{2} & \frac{1}{2} \\
       0 & 0 & \frac{1}{2} & \frac{1}{4} \\
       0 & \frac{1}{4} & 0 & 0
       \end{matrix}
       \right] $$

In [6]:
Q = P[1:end-1,1:end-1]

4×4 Array{Rational{Int64},2}:
 0//1  1//1  0//1  0//1
 0//1  0//1  1//2  1//2
 0//1  0//1  1//2  1//4
 0//1  1//4  0//1  0//1

The stimulus matrix $S$ is as follows.

$$ S = \left[
       \begin{matrix}
       1 & 0 & 0 & 0 & 0 \\
       0 & \frac{1}{2} & \frac{1}{2} & 0 & 0 \\
       0 & \frac{1}{2} & \frac{1}{4} & \frac{1}{4} & 0 \\
       \frac{1}{4} & 0 & 0 & \frac{1}{2} & \frac{1}{4}
       \end{matrix}
       \right] $$

In [7]:
S = [  1    0    0    0    0
       0   1//2 1//2  0    0
       0   1//2 1//4 1//4  0
      1//4  0    0   1//2 1//4 ]

4×5 Array{Rational{Int64},2}:
 1//1  0//1  0//1  0//1  0//1
 0//1  1//2  1//2  0//1  0//1
 0//1  1//2  1//4  1//4  0//1
 1//4  0//1  0//1  1//2  1//4

## Computing the number of occurrences of a state in a test case

The fundamental matrix $N$ is computed as follows.

$$ N = (I - Q)^{-1} $$

In [8]:
N = inv(I-Q)

4×4 Array{Rational{Int64},2}:
 1//1  16//13  16//13  12//13
 0//1  16//13  16//13  12//13
 0//1   2//13  28//13   8//13
 0//1   4//13   4//13  16//13

We compute the matrix of the expected values of the squares $N_2 = \left[\mathrm{E}[n^2_{i,j}]\right]$ as follows.

$$ N_2 = N(2N_d - I) $$

We obtain $N_d$ in Julia from $N$ with `Diagonal(diag(N))`.

In [9]:
Nd = Diagonal(diag(N))
N2 = N*(2Nd - I)

4×4 Array{Rational{Int64},2}:
 1//1  304//169   688//169  228//169
 0//1  304//169   688//169  228//169
 0//1   38//169  1204//169  152//169
 0//1   76//169   172//169  304//169

Now that we can compute the variances directly.

$$ \left[\mathrm{Var}[n_{i,j}]\right]
   = N_2 - N \circ N $$

In [10]:
VarN = N2 - N.*N

4×4 Array{Rational{Int64},2}:
 0//1  48//169  432//169  84//169
 0//1  48//169  432//169  84//169
 0//1  34//169  420//169  88//169
 0//1  60//169   12//13   48//169

In [11]:
convert(Array{Float64},[N[1,:] VarN[1,:]])

4×2 Array{Float64,2}:
 1.0       0.0
 1.23077   0.284024
 1.23077   2.55621
 0.923077  0.497041

The following algorithm computes the terminal expectation and variance matrices, given the single-step matrix $P$.

In [12]:
"""
Compute the non-terminal expectation and
variance matrices for the stochastic matrix
P.

# Examples
```julia-repl
julia> N,V = get_nte(P)
julia> N[1,1:end] V[1,1:end]
```
"""
function get_nte(P::Array)
    Q = P[1:end-1,1:end-1]
    N = inv(I-Q)
    N2 = N * (2*Diagonal(diag(N)) - I)
    V = N2 - N.*N
    return N,V
end

get_nte

Applied to the example matrix $P$ we obtain the expected number of occurences for each state (first column below) and the associated variance (second column below).

In [13]:
N,V = get_nte(P)
convert(Array{Float64},[N[1,:] V[1,:]])

4×2 Array{Float64,2}:
 1.0       0.0
 1.23077   0.284024
 1.23077   2.55621
 0.923077  0.497041

Let $L$ be a random variable counting the number of states visited in a test case.  By summing the expected number of occurrences, we arrive at the expected length of a realization.  Unfortunately we cannot compute the variance this way; we will find it using a different method later.

$$ \mathrm{E}[L] = \sum_{i=1}^n \mathrm{E}[n_{i,j}] $$

In [14]:
convert(Float64,sum(N[1,:]))

4.384615384615385

## Computing the long-run state probabilities

We can compute the long-run occupancies (or long run state probabilities) from $N$ by summing each row and then normalizing (since we can only be in one node at a time).

In [15]:
"""
Compute the Perron eigenvector for the
stochastic matrix P and return it.  The
computation is performed by computation
of the fundamental matrix.

If the fundamental matrix is available,
providing it as the optional second
argument will speed up the computation.

# Examples
```julia-repl
julia> pv = get_pix(P)
julia> pv = get_pix(P,N)
```
"""
function get_pi(P::Array, N::Array=get_nte(P)[1])
    len = 1 + sum(N[1,:])
    pe = N[1:1,:] ./ len
    return [pe 1-sum(pe)]
end

get_pi

In [16]:
pe = @time convert(Array{Float64},get_pi(P))

  0.637102 seconds (1.06 M allocations: 52.234 MiB, 2.82% gc time)


1×5 Array{Float64,2}:
 0.185714  0.228571  0.228571  0.171429  0.185714

This method requires a matrix inversion.  We may be able to compute this faster using the power method, but only if the matrix is primitive.  We can modify the matrix to make it primitive (by adding a self-loop) and then remove the effect of that change in the results.

In [17]:
"""
Compute the Perron eigenvector for the
stochastic matrix P.  The computation is
performed using the power method, and the
approximation and the number of steps are
returned.

An optional initial guess can be provided,
and must be a row array whose length is the
same as the rank of P.

An optional argument `prec` specifies the
required precision, and the optional limit
`limit` specifies the maximum number of
iterations allowed.

```julia-repl
julia> pe, steps = get_pi_approx(P)
julia> pe, steps = get_pi_approx(P, limit=100)
julia> pe, steps = get_pi_approx(P, guess)
```
"""
function get_pi_approx(P::Array,
        guess::Array = ones(1,size(P,1)) .* 1//size(P,1);
        prec=eps(0.0)*10, limit=200)
    n = size(P,1)
    d = n+1
    # Assure primitivity.
    P2 = [P zeros(n,1) ; zeros(1,d)]
    P2[n,d] = 1
    P2[d,d] = 1//2
    P2[d,1] = 1//2
    P2[n,1] = 0
    # Refine the guess.
    yold = [guess guess[end]*2]
    yold = yold ./ sum(yold)
    y = yold * P2
    step = 1
    while sum(abs.(yold - y)) > prec
        step += 1
        if step > limit
            break
        end
        yold = y
        y *= P2
    end
    # Adjust the result to remove the added state.
    y = y[:,1:n]
    y = y ./ sum(y)
    return (y, step)
end

get_pi_approx

In [18]:
pea,steps = @time get_pi_approx(P, prec=0.0001)
println("$(steps) steps")
pea

  2.242230 seconds (4.48 M allocations: 214.017 MiB, 3.80% gc time)
26 steps


1×5 Array{Float64,2}:
 0.185733  0.228563  0.228565  0.171418  0.185721

We can measure the distance from the approximation to the true value.

In [19]:
sum((pea - pe).^2)

6.456462194450325e-10

We can (possibly) improve the guess.

In [20]:
pea,steps = get_pi_approx(P, pea)
sum((pea - pe).^2)

7.703719777548943e-33

## Performing sensitivity analysis

Sensitivity analysis lets us determine which arcs to change to increase flow to specific nodes.  The sensitivity of state $k$ with respect to the transition from state $i$ to state $j$ is defined by the following equation.

$$ z_{i,j,k} = \frac{ \pi_k^{p_{i,j}=0.95} - \pi_k^{p_{i,j}=0.05} }{ 0.90 } $$

To make the change to the matrix, we need to set $p_{i,j}$ to a specific value $v$ and then correct the rest of the values in the row so everything sums to one.  Before making the change to $p_{i,j}$ the row summed to one.  Afterward all other entries (except $i,j$) need to sum to $1 - v$.  To perform the normalization, we need to divide by $1 - p_{i,j}$ and then multiply by $1 - v$.  This happens in the innermost `if` statement in the algorithm below.

In [21]:
"""
Compute the matrix of arc sensitivities.  The first
column of the returned matrix is the source state,
the second column is the target state.  The
remaining columns are the changes in occupancies.

The initial long-run occupancies vector `pe` can be
provided as the optonal second argument.  This
avoids having to compute them before the process
starts.

```julia-repl
julia> sens = get_sensitivities(P)
julia> sens = get_sensitivities(P, pe)
```
"""
function get_sensitivities(P::Array,
        pe::Array = get_pi_approx(P)[1])
    # Loop over all the arcs.
    P2 = P
    n = size(P2,1)
    Z = []
    for i in 1:n
        for j in 1:n
            # If there is an uncertain transition
            # from i to j, compute the sensitivities.
            if P[i,j] != 0 && P[i,j] != 1
                x = 1 - P[i,j] ; t = P[i,1:n]
                P2[i,1:n] = @. t / x * 0.05 ; P2[i,j] = 0.95
                peh, _ = get_pi_approx(P2, pe)
                P2[i,1:n] = @. t / x * 0.95 ; P2[i,j] = 0.05
                pel, _ = get_pi_approx(P2, pe)
                P2[i,1:n] = t
                row = [i j (peh-pel)./0.9]
                if size(Z,1) > 0
                    Z = [Z ; row]
                else
                    Z = row
                end
            end
        end
    end
    return Z
end

get_sensitivities

In [22]:
Z = get_sensitivities(P)

7×7 Array{Float64,2}:
 2.0  3.0  -0.047245    -0.09449     0.37796    -0.18898    -0.047245
 2.0  4.0   0.047245     0.09449    -0.37796     0.18898     0.047245
 3.0  3.0  -0.163906    -0.20173     0.68084    -0.151298   -0.163906
 3.0  4.0   0.00315143   0.0378172  -0.182783    0.138663    0.00315143
 3.0  5.0   0.0806723    0.0645378  -0.161345   -0.0645378   0.0806723
 4.0  2.0  -0.132685     0.0964979   0.0964979   0.0723734  -0.132685
 4.0  5.0   0.132685    -0.0964979  -0.0964979  -0.0723734   0.132685

In [23]:
Z = get_sensitivities(P, pe)

7×7 Array{Float64,2}:
 2.0  3.0  -0.047245    -0.09449     0.37796    -0.18898    -0.047245
 2.0  4.0   0.047245     0.09449    -0.37796     0.18898     0.047245
 3.0  3.0  -0.163906    -0.20173     0.68084    -0.151298   -0.163906
 3.0  4.0   0.00315143   0.0378172  -0.182783    0.138663    0.00315143
 3.0  5.0   0.0806723    0.0645378  -0.161345   -0.0645378   0.0806723
 4.0  2.0  -0.132685     0.0964979   0.0964979   0.0723734  -0.132685
 4.0  5.0   0.132685    -0.0964979  -0.0964979  -0.0723734   0.132685

## Other long-run statistics

In [24]:
"""
Compute the stimulus long run occupancies, given the
stimulus matrix `S` and the long-run state occupancies
`pe`.

```julia-repl
julia> sigma = get_sigma(S, pe)
```
"""
function get_sigma(S::Array, pe)
    # Compute the nomalization factor.
    n = size(S,1) ; m = 1/(1-pe[n+1])
    p2 = pe[:,1:n]
    # Compute the sum for each element.
    return (p2*S).*m
end

get_sigma

In [25]:
sigma = get_sigma(S,pe)

1×5 Array{Float64,2}:
 0.280702  0.280702  0.210526  0.175439  0.0526316

## Probability of occurrence for states

In [26]:
"""
Compute the probability of occurrence
of a state in a realization from its 

```julia-repl
julia> Y = get_node_probability(P)
julia> Y = get_node_probability(P,N)
```
"""
function get_node_probability(P, N = get_nte(P)[1])
    return N * (Diagonal(1 ./ diag(N)))
end

get_node_probability

In [27]:
Y = convert(Array{Float64}, get_node_probability(P, N))

4×4 Array{Float64,2}:
 1.0  1.0    0.571429  0.75
 0.0  1.0    0.571429  0.75
 0.0  0.125  1.0       0.5
 0.0  0.25   0.142857  1.0

## Final notes

In [28]:
varinfo()

| name                 |      size | summary                                                |
|:-------------------- | ---------:|:------------------------------------------------------ |
| A                    | 240 bytes | 5×5 Array{Int64,2}                                     |
| Base                 |           | Module                                                 |
| Core                 |           | Module                                                 |
| Main                 |           | Module                                                 |
| N                    | 296 bytes | 4×4 Array{Rational{Int64},2}                           |
| N2                   | 296 bytes | 4×4 Array{Rational{Int64},2}                           |
| Nd                   | 112 bytes | 4×4 Diagonal{Rational{Int64},Array{Rational{Int64},1}} |
| P                    | 440 bytes | 5×5 Array{Rational{Int64},2}                           |
| Q                    | 296 bytes | 4×4 Array{Rational{Int64},2}                           |
| S                    | 360 bytes | 4×5 Array{Rational{Int64},2}                           |
| V                    | 296 bytes | 4×4 Array{Rational{Int64},2}                           |
| VarN                 | 296 bytes | 4×4 Array{Rational{Int64},2}                           |
| Y                    | 168 bytes | 4×4 Array{Float64,2}                                   |
| Z                    | 432 bytes | 7×7 Array{Float64,2}                                   |
| get_node_probability |   0 bytes | typeof(get_node_probability)                           |
| get_nte              |   0 bytes | typeof(get_nte)                                        |
| get_pi               |   0 bytes | typeof(get_pi)                                         |
| get_pi_approx        |   0 bytes | typeof(get_pi_approx)                                  |
| get_sensitivities    |   0 bytes | typeof(get_sensitivities)                              |
| get_sigma            |   0 bytes | typeof(get_sigma)                                      |
| pe                   |  80 bytes | 1×5 Array{Float64,2}                                   |
| pea                  |  80 bytes | 1×5 Array{Float64,2}                                   |
| row_norm             |   0 bytes | typeof(row_norm)                                       |
| sigma                |  80 bytes | 1×5 Array{Float64,2}                                   |
| steps                |   8 bytes | Int64                                                  |
